In [6]:
import os
import re
import sys
import config
import kmeans
import constants

import numpy as np
import pandas as pd
import plotly.plotly as py
import extract
import transform
import load

from sklearn import cluster
from scipy.stats import zscore

#Pull the main data frame
d_ci = extract.extract_all_lazy()

# Register transform functions here, create them in the transform.py
column_operations = {'ANSWER_num_rooms':transform.answer_num_rooms,
                     'SURVEYTYPE':transform.surveytype_categorical,
                     #'ANSWER_ann_op_rev':transform.answer_ann_op_rev,
                     #'ANSWER_ann_revenue':transform.answer_ann_revenue,
                     'ANSWER_num_employees_pos':transform.answer_num_employees_pos,
                     #This step is dependent on 'create_Executive' to handle weird cases. WW has a better solution?
                     'ANSWER_cur_base_pay':transform.answer_cur_base_pay,
                     'create_Executive':transform.create_Executive
                     #'create_RevPAR':transform.create_RevPAR
                    }

In [5]:
gi = d_ci['gi']
pi = d_ci['pi']
ci = d_ci['ci']
#What Property Information (PI) do I need
pi_features = ['PROPERTY_CODE','STATE','ZIP','CITYMARKET','BINGLATITUDE','BINGLONGITUDE','SURVEYTYPE']
#What General Information (GI) do I need
gi_features = ['PROPERTY_CODE','ANSWER_num_rooms']

#Bake in PI and GI to Compensation Info 
ci = pd.merge(ci, pi[pi_features], on = ['PROPERTY_CODE'])
ci = pd.merge(ci, gi[gi_features], on = ['PROPERTY_CODE'])
#Transformations are ran here, chosen within 'column_operations' and specified in 'transform.py'
ci_payload = transform.clean_model(ci, column_operations)
ci = ci_payload['data']

NameError: name 'd_ci' is not defined

In [3]:
metadata = ci_payload['metadata']
metadata

['SURVEYTYPE_PRIOR :: Rows: 74803, Columns: 23',
 'SURVEYTYPE_POST :: Rows: 74803, Columns: 29',
 'ANSWER_num_employees_pos_PRIOR :: Rows: 74803, Columns: 29',
 'ANSWER_num_employees_pos_POST :: Rows: 74802, Columns: 29',
 'create_Executive_PRIOR :: Rows: 74802, Columns: 29',
 'create_Executive_POST :: Rows: 74802, Columns: 30',
 'ANSWER_num_rooms_PRIOR :: Rows: 74802, Columns: 30',
 'ANSWER_num_rooms_POST :: Rows: 62431, Columns: 30',
 'ANSWER_cur_base_pay_PRIOR :: Rows: 62431, Columns: 30',
 'ANSWER_cur_base_pay_POST :: Rows: 62431, Columns: 32']

In [4]:
ci.columns

Index([           u'PROPERTY_CODE',            u'PROPERTY_NAME',
                       u'JOB_CODE',                 u'POSITION',
            u'ANSWER_cur_base_pay',           u'ANSWER_e_or_ne',
           u'ANSWER_high_base_pay',    u'ANSWER_inc_bonus_comm',
            u'ANSWER_low_base_pay', u'ANSWER_num_employees_pos',
       u'ANSWER_pay_increase_type',           u'ANSWER_pay_max',
                 u'ANSWER_pay_min',         u'ANSWER_start_pay',
              u'ANSWER_target_inc',             u'ANSWER_union',
                          u'STATE',                      u'ZIP',
                     u'CITYMARKET',             u'BINGLATITUDE',
                  u'BINGLONGITUDE',               u'SURVEYTYPE',
               u'ANSWER_num_rooms',            u'Gaming/Casino',
              u'Golf/Country Club',      u'Hospitality - Other',
         u'Lodging - Full Service', u'Lodging - Select Service',
             u'Vacation Ownership',        u'CREATED_executive',
        u'CREATED_base_pa

In [5]:
#Functions to pass for calculations
f = {'POSITION':['count'],
    'CREATED_cur_base_pay_hrs':['mean']}

#ci.groupby(['POSITION','ANSWER_e_or_ne']).agg(f)
ci.columns

Index([           u'PROPERTY_CODE',            u'PROPERTY_NAME',
                       u'JOB_CODE',                 u'POSITION',
            u'ANSWER_cur_base_pay',           u'ANSWER_e_or_ne',
           u'ANSWER_high_base_pay',    u'ANSWER_inc_bonus_comm',
            u'ANSWER_low_base_pay', u'ANSWER_num_employees_pos',
       u'ANSWER_pay_increase_type',           u'ANSWER_pay_max',
                 u'ANSWER_pay_min',         u'ANSWER_start_pay',
              u'ANSWER_target_inc',             u'ANSWER_union',
                          u'STATE',                      u'ZIP',
                     u'CITYMARKET',             u'BINGLATITUDE',
                  u'BINGLONGITUDE',               u'SURVEYTYPE',
               u'ANSWER_num_rooms',            u'Gaming/Casino',
              u'Golf/Country Club',      u'Hospitality - Other',
         u'Lodging - Full Service', u'Lodging - Select Service',
             u'Vacation Ownership',        u'CREATED_executive',
        u'CREATED_base_pa

In [1]:
#Preprocessing step before modeling. Normalize numeric score via Z-score
features = ['CREATED_cur_base_pay_hrs','ANSWER_num_rooms','ANSWER_num_employees_pos']
features = features + ['Gaming/Casino', #Service Types
                                      'Golf/Country Club',
                                      'Lodging - Full Service',
                                      'Lodging - Select Service',
                                      'Vacation Ownership']

features_norm = [f + "_" for f in features]
#ci[features_norm] = ci[features].apply(lambda x: pd.to_numeric(x), axis=1).apply(zscore)

#markets/cities go in 0 position
dimensions = ['CITYMARKET','POSITION']
modeling = ci[features_norm + dimensions]

NameError: name 'ci' is not defined

In [ ]:
modeling

['CREATED_cur_base_pay_hrs',
 'ANSWER_num_rooms',
 'ANSWER_num_employees_pos',
 'Gaming/Casino',
 'Golf/Country Club',
 'Lodging - Full Service',
 'Lodging - Select Service',
 'Vacation Ownership']